In [1]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statistics import mode

import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray
import cupy

from xrspatial import hillshade
from xrspatial import convolution
from datashader.colors import Set1
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.colors import Elevation

from xrspatial import focal, slope
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

In [2]:
# paths
high_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code15_n5.gpkg'
high_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code12_n5.gpkg'
un_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code3_n5.gpkg'
un_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code0_n5.gpkg'
poly_paths = [high_high_path, high_un_path, un_high_path, un_un_path]


helena_path = Path.cwd().parent / 'data' / 'helena'
geomorph_dir = helena_path / 'geomorphons'
crown_path = helena_path / 'crowns'
crown_path_list = [
    c for c
    in crown_path.iterdir()
    if c.suffix == '.gpkg'
    ]



# open treatment polygons
df = pd.concat([gpd.read_file(p) for p in poly_paths])
df = df.drop('area_', axis=1)

We will first check each crown to see if it falls completely within one of the treatment class areas.  If so it will be appended to a datframe of crowns.


In [3]:
# jobs to run in ||
n_jobs = 23

def is_in_treatment(crown_df, row, buf):
    '''Returns only crowns with buffer completely within polygon'''
    crown_df.loc[
        buf.within(row.geometry),
        'treatment'] = row.attribute
    
    return crown_df[crown_df.treatment >= 0 ]


def label_treatment(f):
    crown_df = gpd.read_file(f)
    crown_df = crown_df[crown_df.geometry.area > 10]
    
    # get total bounds of tile as polygon
    bounds = crown_df.total_bounds
    bbox = shapely.geometry.box(*bounds)

    # use only treatment geometries which touch the tile
    sub_df = df[df.geometry.intersects(bbox)]
    if len(sub_df) > 0:
        # add treatment column
        crown_df['treatment'] = -99
        #buffer crowns
        buf = crown_df.geometry.buffer(10)
        # label treatments of crowns lying completely within poly
        return Parallel(n_jobs=n_jobs)(
            delayed(is_in_treatment)(crown_df, row, buf)
            for _, row in sub_df.iterrows()
            )
    else:
        # return empty df, but add treatment column first
        cols = list(crown_df.columns) + ['treatment']
        empty_df = pd.DataFrame(columns=cols)
        return [empty_df]


In [5]:
with joblib_progress('', total=len(crown_path_list)):
    results =  Parallel(n_jobs=n_jobs)(delayed(label_treatment)(f) for f in crown_path_list)
    

Output()

In [7]:
# results is a list of lists of dfs, so we must flatten to concat
crown_df = pd.concat([item for sublist in results for item in sublist])

In [ ]:
    
results = [label_treatment(f) for f in tqdm(crown_path_list)]
crown_df = pd.concat(results)

At this point we will also add a unique identifier.  Then save `crowns_df` so in case ware interrupted,  we will be able to resume without running the 5 hour block of code above again.

In [8]:
def make_unique_ID(crowns, utm_zone):
    '''
    returns copy of dataframe with new uniqueID column
    with entries of form 'utm_zone_x_y where x and y 
    are rounded to the nearest meter.
    TODO: make it round to nearest even meter to lower precision
    '''
    crowns['UniqueID'] = crowns.geometry.centroid.apply(
        lambda p: f'{utm_zone}_{p.x:.0f}_{p.y:.0f}')
    
    return crowns

# add unique ID
crown_df_ = make_unique_ID(crown_df, '10N')
crown_df_.head()

,IDdalponte,zmax,zmean,zsd,zskew,zkurt,zentropy,pzabovezmean,pzabove2,zq5,...,p2th,p3th,p4th,p5th,pground,n,area,geometry,treatment,UniqueID
0,2.0,5.29,4.970000,0.549181,-1.123796,2.307702,0.313845,75.000000,100.0,4.3030,...,0.000000,0.000000,0.0,0.0,0.0,4,0.0672,"POLYGON ((496566.730 4511249.660, 496566.620 4...",12,10N_496567_4511250
1,3.0,5.93,5.684000,0.387337,-1.405252,3.124655,0.000000,80.000000,100.0,5.1560,...,20.000000,0.000000,0.0,0.0,0.0,5,0.0760,"POLYGON ((496570.930 4511249.740, 496570.640 4...",12,10N_496571_4511250
2,4.0,9.26,7.120000,2.037727,-0.419361,1.899452,0.602060,50.000000,100.0,4.8005,...,25.000000,0.000000,0.0,0.0,0.0,4,0.0638,"POLYGON ((496589.250 4511249.710, 496589.000 4...",12,10N_496589_4511250
3,10.0,13.39,11.101111,1.725648,0.357462,1.321602,0.435405,44.444444,100.0,9.4440,...,44.444444,0.000000,0.0,0.0,0.0,9,0.1551,"POLYGON ((496743.460 4511249.750, 496743.330 4...",12,10N_496743_4511250
4,11.0,9.49,6.605000,2.987673,-0.013348,1.025824,0.439247,50.000000,100.0,3.6300,...,50.000000,16.666667,0.0,0.0,0.0,6,0.1148,"POLYGON ((496775.400 4511249.590, 496775.330 4...",12,10N_496775_4511250


In [9]:
# save
crown_df_.to_file(helena_path / 'crowns_with_treatment_label.gpkg')

In [11]:
# make sure the number of treatments is reasonable
crown_df_.treatment.value_counts()

treatment
0     2794929
3      601844
12     468077
15      11980
Name: count, dtype: int64

In [4]:
# now finally we know it is safe to make crown_df = crown_df_ 
#crown_df = crown_df_

# or load it from file if you were interrupted
crown_df = gpd.read_file(helena_path / 'crowns_with_treatment_label.gpkg')

In [6]:
# OMG, I forgot to do this at the begining, probably wasted a lot of time on tiny polys
# fixed above for next time
crown_df = crown_df[crown_df.geometry.area > 10]

In [10]:
# also save fixed version, lets use geoparquet
crown_df.to_parquet(helena_path / 'crowns_with_treatment_label.parquet')

## Geomorphons
In order to look at the effects of slope position on tree mortality we will use the geomorphons algorithm  as implemented in Whitebox Tools.  We will use the geoporphons rasters that were calculated in `src/helena_geomorphon.ipynb`. 

In [2]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statistics import mode

import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray
import cupy

from xrspatial import hillshade
from xrspatial import convolution
from datashader.colors import Set1
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.colors import Elevation

from xrspatial import focal, slope
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

# paths
high_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code15_n5.gpkg'
high_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code12_n5.gpkg'
un_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code3_n5.gpkg'
un_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code0_n5.gpkg'
poly_paths = [high_high_path, high_un_path, un_high_path, un_un_path]


helena_path = Path.cwd().parent / 'data' / 'helena'
geomorph_dir = helena_path / 'geomorphons'
crown_path = helena_path / 'crowns'
crown_path_list = [
    c for c
    in crown_path.iterdir()
    if c.suffix == '.gpkg'
    ]


In [3]:
# or open the parquet,  way faster
crown_df = gpd.read_parquet(helena_path / 'crowns_with_treatment_label.parquet')
crown_df.treatment.value_counts()

treatment
0     2666168
3      527050
12     415553
15       6409
Name: count, dtype: int64

In [4]:
samp_size = 15_000
crown_df = pd.concat(
    [
        crown_df[crown_df.treatment == 15],
        crown_df[crown_df.treatment == 12].sample(samp_size, random_state=1),
        crown_df[crown_df.treatment == 3].sample(samp_size, random_state=1),
        crown_df[crown_df.treatment == 0].sample(samp_size, random_state=1)
        ]
    )

print(f'total: {len(crown_df)}\nValue counts:')
print(crown_df.treatment.value_counts())

total: 51409
Value counts:
treatment
12    15000
3     15000
0     15000
15     6409
Name: count, dtype: int64


In [5]:
for r in tqdm([100, 250, 500, 1000, 2000]):
    # open geomorphon tif
    tif = geomorph_dir / f'geomorph_{r}.tif' 
    gmorph = rioxarray.open_rasterio(tif)
      
    # attach landform to crowns
    centroids = [(c.x, c.y) for c in crown_df.geometry.centroid.to_list()]
    crown_df[f'geomorph_{r}'] = [gmorph.sel(x=x, y=y, method='nearest').item() for x, y in centroids]
    

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [04:37<00:00, 55.43s/it]


Check to make sure all the scales have the same landforms.

In [16]:
geomorph_cols = [col for col in crown_df.columns if 'geomorph' in col]
landforms = set(crown_df[geomorph_cols[0]].to_list())

for col in geomorph_cols:
    v = set(crown_df[col].to_list())
    try:
        assert v == landforms
    except AssertionError:
        print(f'{geomorph_cols[0]} and {col} have different unique landforms')

Now we will create a two-level stratified sample of crowns based on treatment and geomorphon.

In [17]:
def make_samples(r):
    # dict to hold samples
    dict_of_samples = {}

    # split into groups based on treatment
    for tr in [0, 3, 12, 15]:
        df_x = crown_df[crown_df.treatment == tr]

        # for each treatment, split based on landform.
        sub_dict = {}
        for pos in landforms:
            sub_dict[f'geomorph_{r}'] = df_x[df_x[f'geomorph_{r}'] == pos]
        dict_of_samples[f'treatment_{tr}'] = sub_dict

    # find the size of the smallest smallest treatment/landform population
    n = np.inf
    for key1, sub_dict in dict_of_samples.items():
        
        lengths = [len(d) for d in dict_of_samples[key1]]
        n = min(n, min(lengths))
    '''         
    print(f'For {r}, ')    
    print(f'the smallest treatment/landform population is {n}.')
    print(f'It is treatment {tr}')
    '''
    return(dict_of_samples)

    
# dict to hold dicts of samples for each geomorphon scale
dict_of_scales = {}
for r in [100, 250, 500, 1000, 2000]:
    dict_of_scales[f'geomorph_{r}'] = make_samples(r)

In [18]:
rows = []
for r in [100, 250, 500, 1000, 2000]:
    for tr in [0, 3, 12, 15]:
        length = len(dict_of_scales[f'geomorph_{r}'][f'treatment_{tr}'][f'geomorph_{r}'])
        rows.append((r, tr, length))

sample_stats = pd.DataFrame(rows, columns=['scale', 'treatment_code', 'n_samples'])
sample_stats.pivot(index='scale', columns='treatment_code', values='n_samples')

treatment_code,0,3,12,15
scale,,,,
100,100,33,31,0
250,255,82,51,8
500,443,163,113,0
1000,831,290,236,0
2000,1256,551,417,0


In [19]:
samples = {}

for r in [100, 250, 500, 1000, 2000]:
    samps = []
    for tr in [0, 3, 12, 15]:
        # just hardcoded the tr value for sample size based on sample_stats
        sample_size = len(dict_of_scales[f'geomorph_{r}'][f'treatment_{3}'][f'geomorph_{r}'])
        # get desired dataframe
        df_ = dict_of_scales[f'geomorph_{r}'][f'treatment_{tr}'][f'geomorph_{r}']
        # if df_ is larger than sample_size, reduce it to sample_size
        if len(df_) >= sample_size:
            df_ = df_.sample(sample_size, random_state=123)
            
        # stick in list
        samps.append(df_)
    # concat dfs
    samps = pd.concat(samps)
        
    # put samps into the samples dict
    samples[f'geomorph_{r}'] = samps


In [20]:
rows = []
for r in [100, 250, 500, 1000, 2000]:
    df_ = samples[f'geomorph_{r}']
    for tr in [0, 3, 12, 15]:
        length = len(df_[df_.treatment == tr])
        rows.append((r, tr, length))

sample_stats = pd.DataFrame(rows, columns=['scale', 'treatment_code', 'n_samples'])
sample_stats.pivot(index='scale', columns='treatment_code', values='n_samples')

treatment_code,0,3,12,15
scale,,,,
100,33,33,31,0
250,82,82,51,8
500,163,163,113,0
1000,290,290,236,0
2000,551,551,417,0


Next we need to engineer the features for our model, as done in `src/mortality_classification.ipynb`.  Because the Naip imagery for the area is so large, in order for this to run on my machine, the dtype of the imagery has been changed to `np.float32`.  This should not make a difference in the results, but is worth noting.  in `mortality_classification_geographic_holdouts.ipynb` feature creation was run in parallel, but tha tis not possible with sucha large image.

In [21]:
def make_model_inputs(crowns, tif, save_path, y, gk, label=None, IDcolumn=None):
    '''
    Returns DataFrame with features for use in classification model.
    The resulting DataFrame has 'ID' column which matches that in crowns.
    The DataFrame also has a 'label' column, see params for more detail.  

    params:
        crowns   - str - path to OGR readable vector file containing tree crowns.
        tif      - xr  - image used in producing features, already read
                         with rioxarray.
        label    - str - specifies column containing labels.  If specified 'label'
                         column in resulting DataFrame will contain contents of 
                         specified column. Otherwise 'label' column contain -99.
        IDcolumn - str - column to use as matching ID with crowns
    '''
    try:
        # get the extent of the crowns
        xmin, ymin, xmax, ymax = crowns.total_bounds

        # clip the naip image
        print(f'\t\t--clipping...')
        xa = tif.astype(np.float32).rio.clip_box(
            minx=xmin,
            miny=ymin,
            maxx=xmax,
            maxy=ymax
            ).to_dataset(name='band_data')

        # normalized the band_data
        print(f'\t\t--normalizing...')
        band_data = xa.band_data.to_numpy().astype(np.float32)
        band_data = (band_data - np.nanmin(band_data)) * (255 / (np.nanmax(band_data) - np.nanmin(band_data)))

        # calculate relative greenness
        print(f'\t\t--calculating RGI...')
        red = band_data[0]
        green = band_data[1]
        blue = band_data[2]
        nir = band_data[3]
        rgi = green / (red + green + blue)
        xa['rgi'] = (('y', 'x'), rgi)

        # calculate pixel by pixel normalized R, G, B, and NIR
        print(f'\t\t--pix norming...')
        rgbn_tot = red + green + blue + nir
        xa['red_'] = (('y', 'x'), red  / rgbn_tot)
        xa['blue_'] = (('y', 'x'), blue  / rgbn_tot)
        xa['green_'] = (('y', 'x'), green  / rgbn_tot)
        xa['nir_'] = (('y', 'x'), nir  / rgbn_tot)

        # calculate NDVI and SAVI
        print(f'\t\t--NDVI and SAVI...')
        nir_agg = xa.band_data[3].astype(np.float32)
        red_agg = xa.band_data[2].astype(np.float32)
        ndvi_agg = ndvi(nir_agg, red_agg)
        savi_agg = savi(nir_agg, red_agg)
        xa['NDVI'] = ndvi_agg
        xa['SAVI'] = savi_agg

        # calculate RGB luminosity
        print(f'\t\t--luminosity...')
        luminosity = band_data[:3].mean(axis=0) / 255
        xa['luminosity'] = (('y', 'x'), luminosity)

        # mask out shadows and soil for RGI,NDVI, and normed pix colors
        print(f'\t\t--masking...')
        mask = (luminosity > 0.176) & (luminosity < 0.569)
        masked_rgi = xa.rgi.where(mask)
        masked_ndvi = xa.NDVI.where(mask)
        r_ = xa.red_.where(mask)
        g_ = xa.green_.where(mask)
        b_ = xa.blue_.where(mask)
        n_ = xa.nir_.where(mask)
        
        print(f'\t\t--adding index data...')
        data = []
        masked_count = 0
        total = len(crowns)
        bins = np.arange(0.1, 1.1, 0.1)
        with tqdm(total=total) as progress_bar:
            for _, row in crowns.iterrows():
                # calculate luminosity fractions
                lum = xa.luminosity.rio.clip([row.geometry]).to_numpy().flatten()
                lum_tot = lum.shape[0]
                lum_fracs = [((lum < f).sum() - (lum < f - 0.1).sum()) / lum_tot for f in bins]

                # calculate rgi fracs
                rgi = masked_rgi.rio.clip([row.geometry]).to_numpy().flatten()
                rgi = rgi[~np.isnan(rgi)]
                rgi_tot = len(rgi)
                if rgi_tot == 0:
                    rgi_fracs = [-99] * 10
                else:
                    rgi_fracs = [((rgi < f).sum() - (rgi < f - 0.1).sum()) / rgi_tot for f in bins]
                    
                # and normed pix colr fracs
                r = r_.rio.clip([row.geometry]).to_numpy().flatten()
                r = r[~np.isnan(r)]
                c_tot = len(r)
                
                g = g_.rio.clip([row.geometry]).to_numpy().flatten()
                g = g[~np.isnan(g)]

                b = b_.rio.clip([row.geometry]).to_numpy().flatten()
                b = b[~np.isnan(b)]

                n = n_.rio.clip([row.geometry]).to_numpy().flatten()
                n = n[~np.isnan(n)]

                if c_tot == 0:
                    r_fracs = [-99] * 10
                    g_fracs = [-99] * 10
                    b_fracs = [-99] * 10
                    n_fracs = [-99] * 10
                else:
                    r_fracs = [((r < f).sum() - (r < f - 0.1).sum()) / c_tot for f in bins]
                    g_fracs = [((g < f).sum() - (g < f - 0.1).sum()) / c_tot for f in bins]
                    b_fracs = [((b < f).sum() - (b < f - 0.1).sum()) / c_tot for f in bins]
                    n_fracs = [((n < f).sum() - (n < f - 0.1).sum()) / c_tot for f in bins]
                            
                # calculate means and stdevs
                if rgi_tot == 0:
                    ndvi_mean, ndvi_std = -99, -99
                    rgi_mean, rgi_std = -99, -99
                    savi_mean, savi_std = -99, -99
                    r_mean, r_std = -99, -99
                    g_mean, g_std = -99, -99
                    b_mean, b_std = -99, -99
                    n_mean, n_std = -99, -99
                else:
                    #NOTE: .values * 1 casts 1 item DataArray to float
                    ndvi_mean, ndvi_std = masked_ndvi.mean().values * 1, masked_ndvi.std().values * 1
                    rgi_mean, rgi_std = rgi.mean(), rgi.std()
                    savi_mean, savi_std = xa.SAVI.mean().values * 1, xa.SAVI.std().values * 1
                    r_mean, r_std = r.mean(), r.std()
                    g_mean, g_std = g.mean(), g.std()
                    b_mean, b_std = b.mean(), b.std()
                    n_mean, n_std = n.mean(), n.std()

                if label is None:
                    row[label] = -99

                data.append(
                    [row[IDcolumn], (row[label] + 1) / 2] +
                    lum_fracs +
                    rgi_fracs + 
                    r_fracs + 
                    g_fracs + 
                    b_fracs + 
                    n_fracs +
                    [ndvi_mean, ndvi_std, rgi_mean, rgi_std, savi_mean, savi_std] +
                    [r_mean, r_std, g_mean, g_std, b_mean, b_std, n_mean, n_std]
                    )

                #count polygon if has masked pixels            
                if rgi_tot < len(xa.rgi.rio.clip([row.geometry]).to_numpy().flatten()):
                    masked_count = masked_count + 1

                progress_bar.update(1)

        cols = [IDcolumn, 'label',
                'lum10', 'lum20', 'lum30', 'lum40', 'lum50', 'lum60' ,'lum70', 'lum80', 'lum90', 'lum100',
                'rgi10', 'rgi20', 'rgi30', 'rgi40', 'rgi50', 'rgi60' ,'rgi70', 'rgi80', 'rgi90', 'rgi100',
                'r10', 'r20', 'r30', 'r40', 'r50', 'r60' ,'r70', 'r80', 'r90', 'r100',
                'g10', 'g20', 'g30', 'g40', 'g50', 'g60' ,'g70', 'g80', 'g90', 'g100',
                'b10', 'b20', 'b30', 'b40', 'b50', 'b60' ,'b70', 'b80', 'b90', 'b100',
                'n10', 'n20', 'n30', 'n40', 'n50', 'n60' ,'n70', 'n80', 'n90', 'n100',
                'ndvi_mean', 'ndvi_std', 'rgi_mean', 'rgi_std', 'savi_mean', 'savi_std',
                'r_mean', 'r_std', 'g_mean', 'g_std', 'b_mean', 'b_std', 'n_mean', 'n_std']

        data = pd.DataFrame(data, columns=cols)
        data.to_parquet(save_path / f'features_{y}_{gk}.parquet')
        print(y, gk, 'saved to ', str(helena_path / f'features_{y}_{gk}.parquet'))
        del data
    except:
        print(f'OH NO!!! {y}, {gk} FAILED!')

In [23]:
naip_dir = helena_path / 'NAIP'
save_path = helena_path / 'features'
os.makedirs(save_path, exist_ok=True)

def treatment_keys(gk):
        if gk == 250:
            return [0, 3, 12, 15]
        return [3, 12, 15]

features = {}

for y in [2018, 2020, 2022]:
    print(f'----------{y}-----------')                   
    tif_path = naip_dir / str(y) / f'{y}.vrt'
    xa = rioxarray.open_rasterio(tif_path)
    features[y] = {}
    for gk in samples.keys():
        print(f'\t------{gk}/{len(samples.keys())}--------')                   
        features[y][gk] = make_model_inputs(
            samples[gk],
            xa,
            save_path, y, gk,
            label=None,
            IDcolumn='UniqueID'
            )
            

----------2018-----------
	------geomorph_100/5--------
		--clipping...
		--normalizing...
		--calculating RGI...
		--pix norming...
		--NDVI and SAVI...
		--luminosity...
		--masking...
		--adding index data...


  0%|          | 0/97 [00:00<?, ?it/s]

We will use the model which was tuned and trained in `src/mortality_classification.ipynb`.  It was pickled.

In [ ]:
# load model created in src/mortality_classification.ipynb
pickle_path = Path.cwd() / 'RF_model.sav'
model = pickle.load(open(pickle_path, 'rb'))

Now we will make model predictions for the samples and create a timeseries of survival probabilities for each sample over the years for which we have NAIP data.

In [ ]:
treatment_keys = dict_of_samples.keys()
geomorphon_keys = range(1,11)
years = [2018, 2020, 2022]

for tk in treatment_keys:
    for gk in geomorphon_keys:
        ...
def sample_mortality_timeseries(sample_dict, years):
    '''
    Takes a dict of years for a given sample,
    returns a df of probabilities of being alive
    by year.
    '''
    t_series = []
    for y in years:
        cols = sample_dict[y].drop(['y', 'label', 'UniqueID'], axis=1).columns
        X = sample_dict[y][cols]
        lil_df = pd.DataFrame()
        lil_df['UniqueID'] = sample_dict[y]['UniqueID']
        lil_df['pred'] = model.predict_proba(X)[:, 1]
        t_series.append(lil_df)
        
    t_series = [t_series[0].join(df_, on='UniqueID') for df_ in t_series[1:]][0]
        
    return t_series


